# Computing the binary epsilon indicator of two Pareto frontier
This program computes the (additive) binary epsilon indicator of two Pareto-optimal frontiers. This provides a means of comparing the frontiers. See Zitzler 2009 for more.

In [1]:
import pandas as pd
import numpy as np
import time

### Functions used

In [2]:
def computeAdditiveBinaryEpsilonIndicator(f1,f2):
    '''
    Computes the binary epsilon indicator for two frontiers f1 and f2
    The binary epsilon indicator is the addend by which each solution in f1 must be increased in order to be at least 
    as good as every solution in f2
    
    Q: f1 plus what makes it at least as good as f2 everywhere?
    A: epsilon
    '''
        
    # get list of objectives (assumed to be column headers)
    # objectives are assumed to be max
    objectives = f1.columns.tolist()
    
    # initialize the result
    epsilon = -np.Inf
    tc = 0
    for idx2,row2 in f2.iterrows(): # for each solution z2 in f2...
        # what's the minimimum amount that f1 would have to be translated
        # so that at least one solution z1 in f1 weakly dominates z2?
        minTranslationToCover = np.Inf
        for idx1,row1 in f1.iterrows():
            minTranslationToCover = min(minTranslationToCover,max(row2[objectives] - row1[objectives]))
            
        epsilon = max(epsilon,minTranslationToCover)                
    
    return epsilon

In [3]:
def maxAll(origf,objSenses):
    
    f = origf
    
    # Convert each objective to maximization but do not normalize
    for obj,sense in objSenses.items():
        if sense == 0:
            f[obj] = f[obj] * -1
    
    return f

In [4]:
def maxAllAndNorm(origf,objSenses):
    
    f = origf
    
    # get bounds on the objectives' values
    objBounds = {}
    for obj,sense in objSenses.items():
        objBounds[obj] = [f[obj].min(),f[obj].max()]
    # get each objective's ideal value
    idealObjVals = {}
    for obj,bounds in objBounds.items():
        idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max
    # get each objective's worst value
    worstObjVals = {}
    for obj,bounds in objBounds.items():
        worstObjVals[obj] = bounds[not(objSenses[obj])]
        
    # Normalize the objective spaces by converting each value to the relative achievement along its axis:

    #   distance from the worst case value
    # ---------------------------------------
    # total distance spanned by the objective

    # Normalized objective space is in [0,1]^N where N = # of objectives


    for obj,bounds in objBounds.items():
        f[obj] = f[obj].apply(
            lambda x: \
            abs(x - worstObjVals[obj]) / \
            (bounds[1]-bounds[0]))
    
    return f

### Preparation for I_eps computation algorithm

In [5]:
# Get the solutions that define the pareto frontiers.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

f1 = pd.read_csv("../solutionSets/3d/ClimateChange_None/climateChange_EfficientSolutions_NoneOnly.csv")
f2 = pd.read_csv("../solutionSets/3d/ClimateChange_E85/climateChange_EfficientSolutions_E85.csv")

In [6]:
# Hard-coded manipulations to tidy-up sols

# set index, remove unwanted columns
f1 = f1.set_index("1-" + f1["SolutionIndex"].astype(str))
f1 = f1.drop(["SolutionIndex","Frontier"],axis=1)
# set index, remove unwanted columns, rename fire hazard column to match f1, reorder columns
f2 = f2.set_index("2-" + f2["SolutionIndex"].astype(str))
f2 = f2.drop(["SolutionIndex","Frontier"],axis=1).rename(columns={"Fire Hazard Increase":"Fire Hazard"})
f2 = f2[f1.columns.tolist()]

In [7]:
# Get list of objectives from the dataframe (assumed to be identical across the frontiers)

objs = f1.columns.tolist()

In [8]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the dataframe

objSenses = {}

# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0
objSenses["Northern Spotted Owl Habitat (ha)"] = 1
#objSenses["PER"] = 0
#objSenses

### Running the algorithm

In [9]:
# Compute the binary epsilon indicator


# Set this boolean toggle to
    # true if you want to normalize the objective space,
    # false otherwise
normalize = True

if normalize:
    f1 = maxAllAndNorm(f1,objSenses)
    f2 = maxAllAndNorm(f2,objSenses)
else:
    f1 = maxAll(f1,objSenses)
    f2 = maxAll(f2,objSenses)
    
computeAdditiveBinaryEpsilonIndicator(f1,f2)

0.13654458314522833

In [10]:
computeAdditiveBinaryEpsilonIndicator(f2,f1)

0.05736160423470571

### Removing dominated solutions

In [167]:
def removeDominatedSolutions(f):
    '''
    Given a dataframe of solutions, returns a dataframe with only
    nondominated solutions.
    
    It is assumed that each column is an objective and that all objectives
    aim to be maximized.
    '''
    newf = f

    objectives = f.columns.tolist()
    
    for idx,row in f.iterrows(): # searching for a solution that dominates this one
        for idx2,row2 in f.iterrows(): # scanning over all other solutions
            if np.all((row[objectives] < row2[objectives])): # if we find a solution that dominates idx,
                newf = newf.drop(idx) # then we remove that index from the to-be-returned dataframe
                break # and stop searching for a dominating solution
            
    return newf